### Importation des librairies

In [168]:
import mlflow
import mlflow.sklearn
import pickle
import pandas as pd
from mlflow.models.signature import infer_signature
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn import datasets, preprocessing, model_selection, ensemble, pipeline
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
import joblib
import warnings
#Traitement des messages d'erreur :
warnings.filterwarnings('ignore')
import timeit
#Librairie pour gérer l'espace mémoire du PC
import gc
#Librairie pour créer un dataset de train et un de test
from sklearn.model_selection import train_test_split
#Librairie pour l'encodage des variables catégorielles
from sklearn.preprocessing import LabelEncoder
#Librairies pour réaliser un pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
# Importation du package pour le SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline
import math
#Librairie pour XGBoostClassifier
from xgboost import XGBClassifier
#Librairie pour les métriques avec GridSearch
import sklearn.metrics
from sklearn.model_selection import GridSearchCV

In [169]:
#from pycaret.classification import load_model, predict_model

### Fonction utilisées dans le notebook

### Lecture des datasets

In [170]:
path = 'Projet_7/'
data_work = pd.read_csv(path + "data_work.csv")
data_target = pd.read_csv(path + "data_target.csv")
#X_train_res = pd.read_csv(path + "X_train_res.csv")
#_train_res = pd.read_csv(path + "y_train_res.csv")

In [171]:
display(data_work.head())
data_test = data_work.head()

,Type_de_pret,Age,Regime_matrimonial,Nombre_d_enfants,Montant_des_revenus,Montant_des_credits,Note_region_client,Nombre_de_demande_client,Delai_anticipation_pret,Montant_anticipation_pret
0,Prêts de trésorerie,26,célibataire,0,202500,108131.945625,2,1.0,-20.421053,0.0
1,Prêts de trésorerie,46,marié(e),0,270000,254350.125000,1,0.0,-7.000000,0.0
2,Prêts renouvelables,52,célibataire,0,67500,94518.900000,2,0.0,-3.000000,0.0
3,Prêts de trésorerie,52,marié(e),0,135000,NaN,2,NaN,-19.375000,0.0
4,Prêts de trésorerie,55,célibataire,0,121500,146250.000000,2,0.0,-6.320000,0.0


### Lecture du modèle

In [172]:
#Importation du modèle
fichierSauvegarde = open("donnees","rb")
xgb_mod = pickle.load(fichierSauvegarde)
print(xgb_model)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.386,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.289, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)


### Séparation des données

In [173]:
#On commence par diviser le dataset en un dataset de train et un dataset de test avec une proportion de 80% / 20%
X_train, X_test, y_train, y_test = train_test_split(data_work, data_target, test_size = 0.2)
del data_target

In [174]:
X_train.shape, X_test.shape

((246008, 10), (61503, 10))

### Pré-traitement des données

In [175]:
#On remet la variable concernant une période sous un format positif
X_test['Delai_anticipation_pret'] = X_test['Delai_anticipation_pret'].mul(-1)
#On remet la variable concernant une période sous un format positif
X_train['Delai_anticipation_pret'] = X_train['Delai_anticipation_pret'].mul(-1)

#Création d'une variable avec la liste des colonnes catégorielles du dataset features
data_categ = list(data_work.select_dtypes(exclude=["number"]).columns)
#Création d'une variable avec la liste des colonnes numériques du dataset features
data_num = list(data_work.select_dtypes(exclude=["bool_","object_"]).columns)
print("Voici la liste des colonnes catégorielles : \n", data_categ)
print("Voici la liste des colonnes numériques : \n", data_num)
del data_work
#Permet d'initialiser la librairie gc
gc.enable()
#Permet de supprimer les objets qui ne sont plus utilisés
gc.collect()

Voici la liste des colonnes catégorielles : 
 ['Type_de_pret', 'Regime_matrimonial']
Voici la liste des colonnes numériques : 
 ['Age', 'Nombre_d_enfants', 'Montant_des_revenus', 'Montant_des_credits', 'Note_region_client', 'Nombre_de_demande_client', 'Delai_anticipation_pret', 'Montant_anticipation_pret']


2237

In [176]:
#On va maintenant encoder les variables catégorielles
encoder = LabelEncoder()

for col in data_categ:
    X_train[col] = encoder.fit_transform(X_train[col])
    X_test[col] = encoder.fit_transform(X_test[col])
display(X_train)

,Type_de_pret,Age,Regime_matrimonial,Nombre_d_enfants,Montant_des_revenus,Montant_des_credits,Note_region_client,Nombre_de_demande_client,Delai_anticipation_pret,Montant_anticipation_pret
109471,0,39,2,0,207000,NaN,2,NaN,3.094340,0.000000
262249,0,34,2,1,189000,2.480150e+05,2,1.0,9.928571,-1856.931429
20594,0,40,1,0,112500,3.163950e+04,1,1.0,8.142857,-1423.414286
205458,0,51,2,0,270000,4.547464e+05,2,3.0,2.041667,-1721.274375
642,0,39,2,0,157500,1.946871e+06,2,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
138765,0,27,2,0,135000,2.250000e+03,3,2.0,NaN,NaN
280429,0,62,2,0,202500,0.000000e+00,2,0.0,9.250000,0.000000
88254,0,58,2,0,135000,1.680000e+05,2,3.0,4.981132,-974.465660
114237,0,42,2,0,270000,5.077530e+05,2,0.0,15.875000,-674.105625


In [177]:
display(X_train.loc[3])
print(X_train['Regime_matrimonial'].unique())

Type_de_pret                      0.000
Age                              52.000
Regime_matrimonial                2.000
Nombre_d_enfants                  0.000
Montant_des_revenus          135000.000
Montant_des_credits                 NaN
Note_region_client                2.000
Nombre_de_demande_client            NaN
Delai_anticipation_pret          19.375
Montant_anticipation_pret         0.000
Name: 3, dtype: float64

[2 1 0]


In [178]:
scaler = preprocessing.RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Recherche des hyperparamètres

In [179]:
#XGBOOST

xgb = XGBClassifier()

#xgb_parameters = {'colsample_bytree': [0.1, 0.5, 1], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5],
#                  'max_depth': [1, 3, 5, 7, 10]}
#xgb_parameters = {'colsample_bytree': [0.1, 0.5, 1], 'gamma': [0.3],
#                  'max_depth': [5]}
#xgb_parameters = {'colsample_bytree': [0.1], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5],
#                  'max_depth': [5]}
#xgb_parameters = {'colsample_bytree': [0.1], 'gamma': [0.1],
#                  'max_depth': [1, 3, 5, 7, 10]}
#xgb_parameters = {'colsample_bytree': [0.1], 'gamma': [0.1], 'max_depth': [5]}
xgb_parameters = {'max_depth': [5]}

gsv = model_selection.GridSearchCV(xgb, xgb_parameters, cv=5, scoring="accuracy")
#gsv = model_selection.GridSearchCV(xgb, xgb_parameters, cv=5, scoring="roc_auc", "accuracy"])
#gsv = model_selection.GridSearchCV(xgb, xgb_parameters, cv=5)
gsv.fit(X_train_scaled, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max

In [180]:
gsv.best_score_, gsv.best_params_

(0.9190066996220512, {'max_depth': 5})

In [181]:
gsv.best_estimator_.score(X_test_scaled, y_test)

0.9198413085540543

### Pipeline de prédiction

In [182]:
#pipeline = pipeline.Pipeline([('scaler', preprocessing.StandardScaler()), 
#                              ('classifie', ensemble.HistGradientBoostingClassifier(**gsv.best_params_))])

In [183]:
pipeline = pipeline.Pipeline([('scaler', preprocessing.RobustScaler()),
                              ('xgb', ensemble.HistGradientBoostingClassifier(**gsv.best_params_))])

In [184]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', RobustScaler()),
                ('xgb', HistGradientBoostingClassifier(max_depth=5))])

In [185]:
pipeline.score(X_test, y_test)

0.9199713835097475

In [186]:
joblib.dump(pipeline, 'pipeline_housing.joblib')

['pipeline_housing.joblib']

In [188]:

with mlflow.start_run():
    # load dataset and train model
    #iris = load_iris()
    #sk_model = tree.DecisionTreeClassifier()
    #xgb_model_fin = xgb_model.fit(X_train, y_train)
    
    #Temp
    #xgb_model_fin = xgb(xgb_parameters).fit(X_train, y_train)


    # log model params
    #mlflow.log_param("criterion", xgb_model.criterion)
    #mlflow.log_param("splitter", xgb_model.splitter)
    
    signature = infer_signature(X_train, y_train)
    #signature = infer_signature(X_train, xgb_model_fin.predict(X_train))

    # log model
    #Temp
    #mlflow.sklearn.log_model(xgb_model_fin, "xgb_model_final", signature=signature)

In [189]:
#Sauvegare du modèle sklearn
mlflow.sklearn.save_model(xgb_model_fin, 'xgb_model_final', signature=signature)
#mlflow.sklearn.save_model(pipeline, 'mlflow_model', signature=signature)

2023/05/25 12:42:23 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: xgb_model_final\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


In [ ]:
#Temp
#Export des résultats du modèle DummyClassifier
#fichierSauvegarde = open(path + "xgb_model_fin","wb")
#pickle.dump(xgb_model_fin, fichierSauvegarde)
#fichierSauvegarde.close()

In [190]:
print(X_train['Age'].unique())

[39 34 40 51 56 64 53 55 45 59 60 33 36 54 22 50 46 66 48 32 44 47 27 38
 52 28 41 31 61 57 42 30 23 62 65 35 29 58 24 43 37 26 69 49 67 63 21 68
 25 20 70]


In [ ]:
#Résultat de base
#Temps d'entrainement = 46,6ms; AUC = 0,67; Accuracy = 0,88
#XGBoostClassifier : Taux Faux Positifs = 4,66 %; Taux Faux Négatif = 7,05 %

In [ ]:
#Penser à copier le dossier créer xgb_model_final.... dans P7